In [1]:
import pandas as pd
df = pd.read_csv('./onion_for_predict.csv')
print(df.shape)
df = df.drop(['거래월'], axis=1)

(120, 7)


In [2]:
import os

import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn import svm
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint

# df = pd.read_csv('./preprocessed_apartment.csv')
# df.head()
# df.shape
# df = df.sample(n=100)
# print(df.shape)
# df.head()

y = df['가격']
del df['가격']

x_train, x_test,y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=0)

gboost = GradientBoostingRegressor(random_state=0)
xgboost = xgb.XGBRegressor(random_state=0)
lightgbm = lgb.LGBMRegressor(random_state=0)
lasso = Lasso(alpha=1.0, random_state=0)
svm = svm.SVC(kernel='rbf', C=1, gamma=0.1, random_state=0)

models = [{'model':xgboost, 'name':'XGBoost'},
          {'model':gboost, 'name': 'GradientBoosting'},
          {'model':lightgbm, 'name':'LightGBM'},
          {'model':lasso, 'name': 'Lasso'},
          {'model':svm, 'name': 'SVM'}]
def predict_each(models, x, y, sub_x, sub_y):
    for m in models :
        RMSE = []
        
        for m in models:
            start = time.time()
            m['model'].fit(x.values, y)
            predictions = m['model'].predict(sub_x.values) 
            result = sqrt(mean_squared_error(sub_y, predictions))
            end = round((time.time() - start), 4)
            RMSE.append(m['name'])
            RMSE.append(round(result))
            RMSE.append(end)
    return RMSE
y_pred = predict_each(models, x_train, y_train, x_test, y_test)
y_pred

Using TensorFlow backend.
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Future

['XGBoost',
 170,
 0.0312,
 'GradientBoosting',
 162,
 0.0312,
 'LightGBM',
 179,
 0.0156,
 'Lasso',
 205,
 0.0,
 'SVM',
 383,
 0.0156]

In [5]:
model = Sequential()
model.add(Dense(140, input_dim=5, activation='relu'))
model.add(Dense(35, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])

# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
# 모델 저장 조건 설정
modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
# loss는 train loss고 val_loss는 validation loss임 / acc도 마찬가지
# start = time.time()
history = model.fit(x_train, y_train, validation_split=0.4, epochs=100, batch_size=1000, callbacks=[checkpointer])


# # y_vloss에 테스트셋으로 실험 결과의 오차 값을 저장
# y_vloss = history.history['val_loss']

# # y_acc에 학습셋으로 측정한 정확도의 값을 저장
# y_acc = history.history['val_acc']

# # x값을 지정하고 정확도를 파랑색으로, 오차를 빨강색으로 표시
# x_len = np.arange(len(y_acc))
# plt.plot(x_len, y_vloss, "o", c="red", markersize=3)
# plt.plot(x_len, y_acc, "o", c="blue", markersize=3)

# plt.show()

modelPred = model.predict(x_test)

RMSE = sqrt(mean_squared_error(y_test, modelPred))
RMSE

Y_prediction = modelPred.flatten()
for i in range(10):
    label = y_test.values[i]
    prediction = Y_prediction[i]
    print("실제가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction))

Train on 50 samples, validate on 34 samples
Epoch 1/100
50/50 [==============================] - 0s 3ms/step - loss: 21298878464.0000 - accuracy: 0.0000e+00 - val_loss: 1134667.0000 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 1134667.00000, saving model to ./model/01-1134667.0000.hdf5
Epoch 2/100
50/50 [==============================] - 0s 287us/step - loss: 7248858112.0000 - accuracy: 0.0000e+00 - val_loss: 1134668.7500 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 1134667.00000
Epoch 3/100
50/50 [==============================] - 0s 80us/step - loss: 1170030464.0000 - accuracy: 0.0000e+00 - val_loss: 1134670.2500 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 1134667.00000
Epoch 4/100
50/50 [==============================] - 0s 80us/step - loss: 1005254.7500 - accuracy: 0.0000e+00 - val_loss: 1134671.2500 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 1134667.00000
Epoch 5/100
50/50 [=


Epoch 00037: val_loss did not improve from 1134667.00000
Epoch 38/100
50/50 [==============================] - 0s 80us/step - loss: 1005256.8125 - accuracy: 0.0000e+00 - val_loss: 1134671.8750 - val_accuracy: 0.0000e+00

Epoch 00038: val_loss did not improve from 1134667.00000
Epoch 39/100
50/50 [==============================] - 0s 80us/step - loss: 1005256.5625 - accuracy: 0.0000e+00 - val_loss: 1134671.6250 - val_accuracy: 0.0000e+00

Epoch 00039: val_loss did not improve from 1134667.00000
Epoch 40/100
50/50 [==============================] - 0s 80us/step - loss: 1005256.0625 - accuracy: 0.0000e+00 - val_loss: 1134671.2500 - val_accuracy: 0.0000e+00

Epoch 00040: val_loss did not improve from 1134667.00000
Epoch 41/100
50/50 [==============================] - 0s 80us/step - loss: 1005255.9375 - accuracy: 0.0000e+00 - val_loss: 1134670.8750 - val_accuracy: 0.0000e+00

Epoch 00041: val_loss did not improve from 1134667.00000
Epoch 42/100
50/50 [==============================] - 0s 8

Epoch 70/100
50/50 [==============================] - 0s 53us/step - loss: 1005243.3750 - accuracy: 0.0000e+00 - val_loss: 1134657.6250 - val_accuracy: 0.0000e+00

Epoch 00070: val_loss improved from 1134658.12500 to 1134657.62500, saving model to ./model/70-1134657.6250.hdf5
Epoch 71/100
50/50 [==============================] - 0s 97us/step - loss: 1005242.8750 - accuracy: 0.0000e+00 - val_loss: 1134657.2500 - val_accuracy: 0.0000e+00

Epoch 00071: val_loss improved from 1134657.62500 to 1134657.25000, saving model to ./model/71-1134657.2500.hdf5
Epoch 72/100
50/50 [==============================] - 0s 0us/step - loss: 1005242.3750 - accuracy: 0.0000e+00 - val_loss: 1134656.5000 - val_accuracy: 0.0000e+00

Epoch 00072: val_loss improved from 1134657.25000 to 1134656.50000, saving model to ./model/72-1134656.5000.hdf5
Epoch 73/100
50/50 [==============================] - 0s 0us/step - loss: 1005241.9375 - accuracy: 0.0000e+00 - val_loss: 1134656.0000 - val_accuracy: 0.0000e+00

Epoch 0


Epoch 00100: val_loss improved from 1134641.00000 to 1134640.62500, saving model to ./model/100-1134640.6250.hdf5
실제가격: 810.000, 예상가격: 0.013
실제가격: 1247.000, 예상가격: 0.013
실제가격: 1221.000, 예상가격: 0.013
실제가격: 938.000, 예상가격: 0.013
실제가격: 1124.000, 예상가격: 0.013
실제가격: 777.000, 예상가격: 0.013
실제가격: 884.000, 예상가격: 0.013
실제가격: 938.000, 예상가격: 0.013
실제가격: 960.000, 예상가격: 0.013
실제가격: 1070.000, 예상가격: 0.013


In [7]:
model = Sequential()
model.add(Dense(140, input_dim=5, activation='relu'))
model.add(Dense(35, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])

# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
# 모델 저장 조건 설정
modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
# loss는 train loss고 val_loss는 validation loss임 / acc도 마찬가지

# start = time.time()
# history = model.fit(x_train, y_train, validation_split=0.4, epochs=10, batch_size=1000, callbacks=[checkpointer])
# # y_vloss에 테스트셋으로 실험 결과의 오차 값을 저장
# y_vloss = history.history['val_loss']

# # y_acc에 학습셋으로 측정한 정확도의 값을 저장
# y_acc = history.history['val_acc']

# # x값을 지정하고 정확도를 파랑색으로, 오차를 빨강색으로 표시
# x_len = np.arange(len(y_acc))
# plt.plot(x_len, y_vloss, "o", c="red", markersize=3)
# plt.plot(x_len, y_acc, "o", c="blue", markersize=3)

# plt.show()

modelPred = model.predict(x_test)
# end = time.time() - start
# print(round(end,4))
RMSE = sqrt(mean_squared_error(y_test, modelPred))
RMSE

422182.29242169496